<a href="https://colab.research.google.com/github/mmcuervo/ExamMLcategorias/blob/Experimento3/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook2 DATA ENGENIERING: En este archivo transformamos el data set**

**Problema:** El equipo de atención al cliente recibe cerca de 1000 tickets por día, algunos son urgentes y otros no, necesitamos un sistema que clasifique los tickets por prioridad, alta, media y baja. Para que se atiendan los de prioridad alta primero.

**Pregunta que responde el modelo:** ¿Cuál es la prioridad de el ticket analizado?

**Pregunta que responde este notebook:** ¿Los datos de entrada son acordes con el dataset de entrenamiento?

**Acciones a realizar:**  

1.   Modifacar el dataSet de acuerdo a las reglas del EDA para que los datos de producción sean homogéneos a los datos de entrenamiento.
2.   Crear un informe log, sobre la calidad de los datos recibidos.

**CONTRATO DE DATOS:**

*Variables necesarias para analizar el modelo:*

 **Variables categóricas:** company_size,industry, customer_tier, product_area
** Variables numéricas (int64):** past_30d_tickets, past_90d_incidents,description_length, customers_affected,downtime_min
** Variables numéricas (float64):** error_rate_pct
 **Variables booleanas (int64):** payment_impact_flag, data_loss_flag, security_incident_flag, has_runbook  

**Tratamiento de data_type erroneo:**
Todos los casos: Clasificarlo como nulo (UKNOWN para texto , 0 para numeros) y tratarlos como nulos

***Reglas Datos nulos***

Tratamiento de datos nulos:

**Categorías nominales:** Se registra **UNKNOWN** y se clasifican como **other**
**Categorías numéricas:** Imputar con mediana
**Categoría boolena:** Remplazar nulo con 0

Siempre registrar un Log de los datos nulos para reporte de calidad de dataSet Interpretación de manejo de datos nulos que superen el valor de entrenamiento:** 0–5:%** ✅ aceptable. **5–10:% ** ⚠️ investigar. >10:%🚨 problema de calidad. Datos nulos en customer_sentment =1.8%, resto de las variables 0%

**Tratamiento de datos no previstos**
**Datos categóricos:** Si es texto se respeta el feature pero se clasifica como other, si es número se registra como **UNKNOWN** y se clasifica como other
Datos booleanos:Si es texto se decalara como nulo y se aplica la regla de dato nulo, si es número (ej: 3) se remplaza con cero

Siempre registrar un Log de los datos nulos para reporte de calidad de dataSet
Interpretación de manejo de datos nulos que superen el valor de entrenamiento**:0-5:**% ✅ aceptable.**5-10**:%  ⚠️ investigar. **>10**:% 🚨 problema de calidad. Datos nulos en customer_sentment =**1.8**%, resto de las variables **0**%**

**Criterios de escalamiento**

**Datos numéricos:**
Aplicar Log-transform (log1p) + Z-score a:
downtime_min, customer_affected y error_rate_pct.
Eliminar columnas originales quedarse con las de sufijo _log
Escalar con Z-score a:
past_30d_tickets, past_90d_incidents, y description_length.

Datos categóricos y booleanos no se escalan
Reportar min y max  de los datos en un log para análisis de calidad de los datos

**Criterio de diseño de dataset final:**
Exportar el dataset final siguiendo los lineamientos de Tidy Data

# **Ingesta del DataSet**

In [1]:
# 1) Importar librerías básicas para EDA
import pandas as pd              # Manejo de datos en DataFrames
import numpy as np               # Utilidades numéricas (vectores, matrices, NaN)
import matplotlib.pyplot as plt  # Gráficas base
import seaborn as sns            # Gráficas estadísticas (opcional, solo EDA)
from pathlib import Path         # Manejo seguro de rutas de archivos


In [2]:
#2 Llamamos al dataset y creamos una copia para trabajar en ella y no tocar el original
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/AiLabData/Support_tickets.csv"
df = pd.read_csv(DATA_PATH, encoding="utf-8")
dc = df.copy()

Mounted at /content/drive


In [4]:
#3) Verificar que el dataset esté completo y eliminar columnas sobrantes

# Definir las columnas esperadas y sus tipos de datos basados en el contrato
expected_columns = {
    'company_size': 'object',
    'industry': 'object',
    'customer_tier': 'object',
    'product_area': 'object',
    'past_30d_tickets': 'int64',
    'past_90d_incidents': 'int64',
    'description_length': 'int64',
    'customers_affected': 'int64',
    'error_rate_pct': 'float64',
    'downtime_min': 'int64',
    'payment_impact_flag': 'int64',
    'data_loss_flag': 'int64',
    'security_incident_flag': 'int64',
    'has_runbook': 'int64'
}

# Verificar si faltan columnas
missing_columns = [col for col in expected_columns.keys() if col not in dc.columns]

if missing_columns:
    # Si faltan columnas, imprimir mensaje de error y registrar en un log
    error_message = f"Lo siento, el dataset está incompleto. Faltan las siguientes columnas: {', '.join(missing_columns)}. No es posible continuar con el análisis."
    print(error_message)

    # Crear un archivo de log
    log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")
    with open(log_file_path, "w") as log_file:
        log_file.write("Reporte de Columnas Faltantes:\n")
        for col in missing_columns:
            log_file.write(f"- {col}\n")
    print(f"Se ha creado un archivo de log en: {log_file_path}")

else:
    print("Todas las columnas esperadas están presentes.")

    # Eliminar las columnas que no están en la lista de columnas esperadas
    columns_to_drop = [col for col in dc.columns if col not in expected_columns.keys()]
    dc.drop(columns=columns_to_drop, inplace=True)

    print("\nColumnas restantes en el DataFrame después de eliminar las no esperadas:")
    print(dc.columns.tolist())

    # Mostrar información y primeras filas del DataFrame resultante
    print("\nInformación del DataFrame después de la validación y limpieza:")
    dc.info()
    print("\nPrimeras filas del DataFrame después de la validación y limpieza:")
    display(dc.head())

Todas las columnas esperadas están presentes.

Columnas restantes en el DataFrame después de eliminar las no esperadas:
['company_size', 'industry', 'customer_tier', 'product_area', 'past_30d_tickets', 'past_90d_incidents', 'description_length', 'customers_affected', 'error_rate_pct', 'downtime_min', 'payment_impact_flag', 'data_loss_flag', 'security_incident_flag', 'has_runbook']

Información del DataFrame después de la validación y limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   product_area            50000 non-null  object 
 4   past_30d_tickets        50000 non-null  Int64  
 5   past_90d_incidents      50000 non-null  Int64  
 6   description_le

,company_size,industry,customer_tier,product_area,past_30d_tickets,past_90d_incidents,description_length,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,data_loss_flag,security_incident_flag,has_runbook
0,Small,media,Basic,mobile,2,0,227,2,5.451201,6,0,0,0,0
1,Small,healthcare,Basic,analytics,2,3,461,24,2.495538,2,0,0,0,0
2,Small,gaming,Basic,notifications,3,1,306,1,3.328402,0,0,0,0,1
3,Small,media,Plus,analytics,2,2,363,25,2.931906,16,0,0,0,1
4,Small,ecommerce,Plus,analytics,5,1,442,28,3.255222,6,0,0,0,0


In [6]:
#4 Estandarización del DataSet, asegurar que los features correspondan al tipo de dato esperado

# Path to the log file
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")

# Dictionary to store corrected values count per column
corrected_values_count = {}

print("\nVerificando y corrigiendo tipos de datos:")
# Checking and correcting data types:
for col, expected_dtype in expected_columns.items():
    if col in dc.columns:
        initial_dtype = dc[col].dtype
        initial_non_null_count = dc[col].notna().sum()
        unparsable_count = 0 # Inicializar unparsable_count a 0 al comienzo de cada iteración

        try:
            if expected_dtype == 'object':
                # If expected is object, ensure it's treated as object
                dc[col] = dc[col].astype(str)
            elif expected_dtype in ['int64', 'float64']:
                # Clean numerical columns: remove non-digit characters except dot for float
                if expected_dtype == 'int64':
                     dc[col] = dc[col].astype(str).str.replace(r'[^\d]', '', regex=True)
                elif expected_dtype == 'float64':
                     dc[col] = dc[col].astype(str).str.replace(r'[^\d.]', '', regex=True)

                # Convert to numeric, coercing errors. Invalid parsing will result in NaN.
                numeric_series = pd.to_numeric(dc[col], errors='coerce')

                # Count how many values were not originally NaN but became NaN after coercion
                # This indicates values that couldn't be parsed as numbers
                unparsable_count = dc[col].notna().sum() - numeric_series.notna().sum()

                # Fill NaN values (including those from coercion) with 0
                dc[col] = numeric_series.fillna(0)

                # Convert to the specific numeric type, handling nullable types
                if expected_dtype == 'int64':
                    dc[col] = dc[col].astype('Int64')
                elif expected_dtype == 'float64':
                    dc[col] = dc[col].astype('Float64')

            # Calculate the number of values that changed type or were filled with 0
            # This is a simpler way to estimate corrections after the fill/coerce steps
            # We can't easily track exact "corrections" per original value without more complex logic,
            # so we'll report on values that became 0 due to being unparsable.
            if unparsable_count > 0:
                 corrected_values_count[col] = unparsable_count


        except Exception as e:
            print(f"Error processing column {col}: {e}")


# Report on corrected values and write to log
print("\nReporte de corrección de tipos de datos y valores:")
# Report on data type and value correction:
if corrected_values_count:
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Corrección de Tipos de Datos y Valores:\n")
        for col, count in corrected_values_count.items():
            message = f"- Columna '{col}': {count} valores que no pudieron ser convertidos a numérico se corrigieron a 0."
            print(message)
            log_file.write(message + "\n")
else:
    message = "No se encontraron valores no numéricos que requirieran corrección a 0 en las columnas numéricas esperadas."
    print(message)
    with open(log_file_path, "a") as log_file:
         log_file.write("\n" + message + "\n")


# Verify data types after correction
dtype_errors_after = []
for col, expected_dtype in expected_columns.items():
    if col in dc.columns:
        if dc[col].dtype.name != expected_dtype:
             # Handle Int64 vs int64 and Float64 vs float64
            if not ((expected_dtype == 'int64' and dc[col].dtype.name == 'Int64') or
                    (expected_dtype == 'float64' and dc[col].dtype.name == 'Float64')):
                dtype_errors_after.append(f"La columna '{col}' tiene el tipo de dato '{dc[col].dtype.name}' pero se esperaba '{expected_dtype}' después de la corrección.")

if dtype_errors_after:
    print("\nErrores en los tipos de datos después de la corrección:")
    # Print data type errors after correction if they exist:
    for error in dtype_errors_after:
        print(error)
        with open(log_file_path, "a") as log_file:
            log_file.write(error + "\n")
else:
    print("\nTodos los tipos de datos de las columnas esperadas coinciden con los definidos después de la corrección.")
    # All expected column data types match the defined ones after correction.


# Display info and head of the resulting DataFrame
# Mostrar información y primeras filas del DataFrame resultante
print("\nInformación del DataFrame después de la validación y corrección de tipos:")
dc.info()
print("\nPrimeras filas del DataFrame después de la validación y corrección de tipos:")
display(dc.head())


Verificando y corrigiendo tipos de datos:

Reporte de corrección de tipos de datos y valores:
No se encontraron valores no numéricos que requirieran corrección a 0 en las columnas numéricas esperadas.

Todos los tipos de datos de las columnas esperadas coinciden con los definidos después de la corrección.

Información del DataFrame después de la validación y corrección de tipos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   product_area            50000 non-null  object 
 4   past_30d_tickets        50000 non-null  Int64  
 5   past_90d_incidents      50000 non-null  Int64  
 6   description_length      50000 non-null  Int64  
 7   customers_affected      50

,company_size,industry,customer_tier,product_area,past_30d_tickets,past_90d_incidents,description_length,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,data_loss_flag,security_incident_flag,has_runbook
0,Small,media,Basic,mobile,2,0,227,2,5.451201,6,0,0,0,0
1,Small,healthcare,Basic,analytics,2,3,461,24,2.495538,2,0,0,0,0
2,Small,gaming,Basic,notifications,3,1,306,1,3.328402,0,0,0,0,1
3,Small,media,Plus,analytics,2,2,363,25,2.931906,16,0,0,0,1
4,Small,ecommerce,Plus,analytics,5,1,442,28,3.255222,6,0,0,0,0


In [7]:
#5 Manejo de datos nulos

# Path to the log file
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")

print("\nManejo de datos nulos:")
# Handling null values:

# Dictionary to store the number of null values found and corrected per column
null_correction_report = {}

# Iterate over columns to handle null values
for col in dc.columns:
    if dc[col].isnull().sum() > 0:
        # Count nulls before correction
        initial_null_count = dc[col].isnull().sum()

        if dc[col].dtype == 'object':
            # Replace nulls with 'UNKNOWN' for object type columns
            dc[col].fillna('UNKNOWN', inplace=True)
            corrected_count = initial_null_count - dc[col].isnull().sum()
            if corrected_count > 0:
                null_correction_report[col] = {'initial_nulls': initial_null_count, 'corrected_to': 'UNKNOWN', 'corrected_count': corrected_count}
        elif pd.api.types.is_numeric_dtype(dc[col]):
            # Replace nulls with 0 for numerical columns
            dc[col].fillna(0, inplace=True)
            corrected_count = initial_null_count - dc[col].isnull().sum()
            if corrected_count > 0:
                null_correction_report[col] = {'initial_nulls': initial_null_count, 'corrected_to': 0, 'corrected_count': corrected_count}

# Report on null value corrections and write to log
print("\nReporte de corrección de datos nulos:")
# Report on null value correction:
if null_correction_report:
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Corrección de Datos Nulos:\n")
        for col, report_data in null_correction_report.items():
            message = (f"- Columna '{col}': Se encontraron {report_data['initial_nulls']} valores nulos. "
                       f"Se corrigieron {report_data['corrected_count']} valores a '{report_data['corrected_to']}'.")
            print(message)
            log_file.write(message + "\n")
else:
    message = "No se encontraron valores nulos que requirieran corrección."
    print(message)
    with open(log_file_path, "a") as log_file:
         log_file.write("\n" + message + "\n")

# Verify no more null values in the expected columns
print("\nVerificación de valores nulos después de la corrección:")
# Verification of null values after correction:
nulls_after_correction = dc.isnull().sum()
if nulls_after_correction.sum() == 0:
    print("No se encontraron valores nulos después de la corrección.")
else:
    print("Columnas con valores nulos después de la corrección:")
    display(nulls_after_correction[nulls_after_correction > 0])

# Display info and head of the resulting DataFrame
print("\nInformación del DataFrame después del manejo de datos nulos:")
dc.info()
print("\nPrimeras filas del DataFrame después del manejo de datos nulos:")
display(dc.head())


Manejo de datos nulos:

Reporte de corrección de datos nulos:
No se encontraron valores nulos que requirieran corrección.

Verificación de valores nulos después de la corrección:
No se encontraron valores nulos después de la corrección.

Información del DataFrame después del manejo de datos nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   product_area            50000 non-null  object 
 4   past_30d_tickets        50000 non-null  Int64  
 5   past_90d_incidents      50000 non-null  Int64  
 6   description_length      50000 non-null  Int64  
 7   customers_affected      50000 non-null  Int64  
 8   error_rate_pct          50000 non-null  Float64
 9   dow

,company_size,industry,customer_tier,product_area,past_30d_tickets,past_90d_incidents,description_length,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,data_loss_flag,security_incident_flag,has_runbook
0,Small,media,Basic,mobile,2,0,227,2,5.451201,6,0,0,0,0
1,Small,healthcare,Basic,analytics,2,3,461,24,2.495538,2,0,0,0,0
2,Small,gaming,Basic,notifications,3,1,306,1,3.328402,0,0,0,0,1
3,Small,media,Plus,analytics,2,2,363,25,2.931906,16,0,0,0,1
4,Small,ecommerce,Plus,analytics,5,1,442,28,3.255222,6,0,0,0,0


In [8]:
#6 Aplicamos log1p(x) a 'customers_affected', error_rate_pct, and 'downtime_min'
# Applying log1p(x) to 'customers_affected', error_rate_pct, and 'downtime_min'
cols_to_log = ['customers_affected', 'downtime_min', 'error_rate_pct']

for col in cols_to_log:
    new_col_name = f'{col}_log'
    # Ensure column exists and is numeric before applying log1p
    if col in dc.columns and pd.api.types.is_numeric_dtype(dc[col]):
        dc[new_col_name] = np.log1p(dc[col])
        print(f"Log1p transformación aplicada a '{col}', resultado guardado en '{new_col_name}'.")
    else:
        print(f"Advertencia: La columna '{col}' no se encontró o no es numérica. No se aplicó log1p.")


# Eliminar las columnas originales después de la transformación
# Drop the original columns after transformation
dc.drop(columns=cols_to_log, inplace=True, errors='ignore')
print(f"\nColumnas originales eliminadas: {cols_to_log}")


print("\nPrimeras 5 filas con las nuevas columnas log:")
# Display first 5 rows with the new log columns
log_cols_created = [f'{col}_log' for col in cols_to_log if f'{col}_log' in dc.columns]
if log_cols_created:
    display(dc[log_cols_created].head())
else:
    print("No se crearon columnas log.")

print("\nValores máximos y mínimos de las columnas log transformadas:")
# Display max and min values of the log-transformed columns
if log_cols_created:
    display(dc[log_cols_created].agg(['min', 'max']))
else:
     print("No hay columnas log para mostrar min/max.")

# Mostrar información del DataFrame para verificar las columnas
# Display DataFrame info to verify columns
print("\nInformación del DataFrame después de la transformación y eliminación:")
dc.info()

Log1p transformación aplicada a 'customers_affected', resultado guardado en 'customers_affected_log'.
Log1p transformación aplicada a 'downtime_min', resultado guardado en 'downtime_min_log'.
Log1p transformación aplicada a 'error_rate_pct', resultado guardado en 'error_rate_pct_log'.

Columnas originales eliminadas: ['customers_affected', 'downtime_min', 'error_rate_pct']

Primeras 5 filas con las nuevas columnas log:


,customers_affected_log,downtime_min_log,error_rate_pct_log
0,1.098612,1.94591,1.864266
1,3.218876,1.098612,1.251487
2,0.693147,0.0,1.465198
3,3.258097,2.833213,1.369124
4,3.367296,1.94591,1.448147



Valores máximos y mínimos de las columnas log transformadas:


,customers_affected_log,downtime_min_log,error_rate_pct_log
min,0.000000,0.000000,0.000000
max,8.658345,5.283204,4.392157



Información del DataFrame después de la transformación y eliminación:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   product_area            50000 non-null  object 
 4   past_30d_tickets        50000 non-null  Int64  
 5   past_90d_incidents      50000 non-null  Int64  
 6   description_length      50000 non-null  Int64  
 7   payment_impact_flag     50000 non-null  Int64  
 8   data_loss_flag          50000 non-null  Int64  
 9   security_incident_flag  50000 non-null  Int64  
 10  has_runbook             50000 non-null  Int64  
 11  customers_affected_log  50000 non-null  Float64
 12  downtime_min_log        50000 non-null  Float64
 13  error_rate_pct_log  

In [10]:
#7 aplicmo z-score a variabls numericas
from sklearn.preprocessing import StandardScaler

cols_to_scale = [
    "past_30d_tickets",
    "past_90d_incidents",
    "description_length",
    "downtime_min_log",
    "customers_affected_log",
    "error_rate_pct_log"
]

# Iniciamos StandardScaler
scaler = StandardScaler()

 #Aplicamos la escalación estsndard (z-score)
dc[cols_to_scale] = scaler.fit_transform(dc[cols_to_scale])

print("Datos escalados (primeras 5 filas):")
display(dc[cols_to_scale].head())

print("\nValores Min and Max después de escalar:")
display(dc[cols_to_scale].agg(['min', 'max']))

Datos escalados (primeras 5 filas):


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
0,-0.652358,-1.257509,-1.368881,0.345205,-1.387750,0.553666
1,-0.652358,0.786423,0.874317,-0.206401,-0.228376,-0.302346
2,-0.293185,-0.576198,-0.611562,-0.921618,-1.609461,-0.003806
3,-0.652358,0.105113,-0.065142,0.922856,-0.206930,-0.138015
4,0.425160,-0.576198,0.692177,0.345205,-0.147219,-0.027626



Valores Min and Max después de escalar:


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
min,-1.370703,-1.257509,-3.353248,-0.921618,-1.988478,-2.050592
max,5.453574,6.918218,4.267873,2.517845,2.745960,4.084965


In [ ]:
#3 Verificamos el archivo (info, head)
print("Dimensiones del dataset:" )
dc.info()
pd.set_option('display.max_columns', None)
dc.head()

Dimensiones del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ticket_id               50000 non-null  int64  
 1   day_of_week             50000 non-null  object 
 2   day_of_week_num         50000 non-null  int64  
 3   company_id              50000 non-null  int64  
 4   company_size            50000 non-null  object 
 5   company_size_cat        50000 non-null  int64  
 6   industry                50000 non-null  object 
 7   industry_cat            50000 non-null  int64  
 8   customer_tier           50000 non-null  object 
 9   customer_tier_cat       50000 non-null  int64  
 10  org_users               50000 non-null  int64  
 11  region                  50000 non-null  object 
 12  region_cat              50000 non-null  int64  
 13  past_30d_tickets        50000 non-null  int64  
 14  past_90d_inci

,ticket_id,day_of_week,day_of_week_num,company_id,company_size,company_size_cat,industry,industry_cat,customer_tier,customer_tier_cat,org_users,region,region_cat,past_30d_tickets,past_90d_incidents,product_area,product_area_cat,booking_channel,booking_channel_cat,reported_by_role,reported_by_role_cat,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,customer_sentiment,customer_sentiment_cat,description_length,priority,priority_cat
0,1000000000,Wed,3,100015,Small,1,media,7,Basic,1,126,APAC,3,2,0,mobile,3,web,1,support,1,2,5.451201,6,0,0,0,0,neutral,2,227,low,1
1,1000000001,Sat,6,100023,Small,1,healthcare,5,Basic,1,101,AMER,1,2,3,analytics,5,chat,3,product_manager,3,24,2.495538,2,0,0,0,0,neutral,2,461,low,1
2,1000000002,Mon,1,100012,Small,1,gaming,4,Basic,1,71,APAC,3,3,1,notifications,6,chat,3,devops,2,1,3.328402,0,0,0,0,1,positive,3,306,low,1
3,1000000003,Wed,3,100003,Small,1,media,7,Plus,2,100,AMER,1,2,2,analytics,5,chat,3,finance,4,25,2.931906,16,0,0,0,1,neutral,2,363,medium,2
4,1000000004,Mon,1,100019,Small,1,ecommerce,2,Plus,2,332,AMER,1,5,1,analytics,5,web,1,support,1,28,3.255222,6,0,0,0,0,neutral,2,442,low,1


In [ ]:
#4 Normalizamos los datatype Int64 y float64
column=[ 'ticket_id', 'day_of_week_num','company_id',
  'company_size_cat', 'industry_cat', 'customer_tier_cat',
  'org_users','region_cat', 'past_30d_tickets',
  'past_90d_incidents', 'product_area_cat', 'booking_channel_cat',
  'reported_by_role_cat','customers_affected','downtime_min',
  'payment_impact_flag', 'security_incident_flag', 'data_loss_flag',
  'has_runbook', 'customer_sentiment_cat','description_length',
  'priority_cat','error_rate_pct']

# Aplicamos numeric y astype a cada columna
for col in column:
  col_dtype = dc[col].dtype
  if pd.api.types.is_integer_dtype(col_dtype):
    dc[col] = pd.to_numeric(dc[col], errors="coerce").astype("Int64")
  else:
    dc[col] = pd.to_numeric(dc[col], errors="coerce").astype("Float64")

# **Escalamiento de variables númericas**

**Conclusión:** Eliminar columnas originales de: customers_affected, downtime_min, error_rate_pct

In [ ]:
#20 aplicmo z-score a variabls numericas
from sklearn.preprocessing import StandardScaler

cols_to_scale = [
    "past_30d_tickets",
    "past_90d_incidents",
    "description_length",
    "downtime_min_log",
    "customers_affected_log",
    "error_rate_pct_log"
]

# Iniciamos StandardScaler
scaler = StandardScaler()

 #Aplicamos la escalación estsndard (z-score)
dc[cols_to_scale] = scaler.fit_transform(dc[cols_to_scale])

print("Datos escalados (primeras 5 filas):")
display(dc[cols_to_scale].head())

print("\nValores Min and Max después de escalar:")
display(dc[cols_to_scale].agg(['min', 'max']))

Datos escalados (primeras 5 filas):


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
0,-0.652358,-1.257509,-1.368881,0.345205,-1.387750,0.553666
1,-0.652358,0.786423,0.874317,-0.206401,-0.228376,-0.302346
2,-0.293185,-0.576198,-0.611562,-0.921618,-1.609461,-0.003806
3,-0.652358,0.105113,-0.065142,0.922856,-0.206930,-0.138015
4,0.425160,-0.576198,0.692177,0.345205,-0.147219,-0.027626



Valores Min and Max después de escalar:


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
min,-1.370703,-1.257509,-3.353248,-0.921618,-1.988478,-2.050592
max,5.453574,6.918218,4.267873,2.517845,2.745960,4.084965


**Conclusión:** Eliminamos region

## Manejo de variables categóricas, (tidydata)

In [ ]:
# 28Apply one-hot encoding to categorical variables
cols_to_encode = ['industry', 'company_size', 'customer_tier', 'product_area']

# Use pd.get_dummies to apply one-hot encoding
dc = pd.get_dummies(dc, columns=cols_to_encode, drop_first=True, dtype=int)

print("DataFrame after one-hot encoding:")
dc.info()
display(dc.head())

DataFrame after one-hot encoding:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   past_30d_tickets            50000 non-null  float64 
 1   past_90d_incidents          50000 non-null  float64 
 2   booking_channel             50000 non-null  object  
 3   payment_impact_flag         50000 non-null  int64   
 4   security_incident_flag      50000 non-null  int64   
 5   data_loss_flag              50000 non-null  int64   
 6   has_runbook                 50000 non-null  int64   
 7   description_length          50000 non-null  float64 
 8   priority                    50000 non-null  category
 9   customers_affected_log      50000 non-null  float64 
 10  downtime_min_log            50000 non-null  float64 
 11  error_rate_pct_log          50000 non-null  float64 
 12  industry_fintech            50000 non-nu

,past_30d_tickets,past_90d_incidents,booking_channel,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority,customers_affected_log,downtime_min_log,error_rate_pct_log,industry_fintech,industry_gaming,industry_healthcare,industry_logistics,industry_media,industry_saas_b2b,company_size_Medium,company_size_Small,customer_tier_Enterprise,customer_tier_Plus,product_area_auth,product_area_billing,product_area_data_pipeline,product_area_mobile,product_area_notifications
0,-0.652358,-1.257509,web,0,0,0,0,-1.368881,low,-1.387750,0.345205,0.553666,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1,-0.652358,0.786423,chat,0,0,0,0,0.874317,low,-0.228376,-0.206401,-0.302346,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,-0.293185,-0.576198,chat,0,0,0,1,-0.611562,low,-1.609461,-0.921618,-0.003806,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
3,-0.652358,0.105113,chat,0,0,0,1,-0.065142,medium,-0.206930,0.922856,-0.138015,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,0.425160,-0.576198,web,0,0,0,0,0.692177,low,-0.147219,0.345205,-0.027626,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


# Eliminar variables sobrantes

In [ ]:
# Eliminar la columna 'booking_channel'
delete=["ticket_id",
        "company_size_cat",
        "industry_cat",
        "customer_tier_cat",
        "region_cat",
        "product_area_cat",
        "booking_channel_cat",
        "reported_by_role_cat",
        "customer_sentiment_cat",
        "priority_cat",
        "day_of_week_num",
        "customers_affected",
        "downtime_min",
        "error_rate_pct",
        'company_id',
        'org_users',
        'region',
        'day_of_week',
        'customer_sentiment',
        'reported_by_role',
        'booking_channel']


dc.drop(delete, inplace=True)

print("Columnas restantes en el DataFrame dc después de eliminar variables:")
print(dc.columns.tolist())

print("\nNúmero de subcategorías por columna categórica:")

# Identificar columnas categóricas restantes
categorical_cols_remaining = dc.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols_remaining:
    print(f"\n--- Columna: {col} ---")
    print(f"Número de subcategorías: {dc[col].nunique()}")
    print("Subcategorías:")
    display(dc[col].value_counts().sort_index())

Columnas restantes en el DataFrame dc después de eliminar booking_channel:
['past_30d_tickets', 'past_90d_incidents', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook', 'description_length', 'priority', 'customers_affected_log', 'downtime_min_log', 'error_rate_pct_log', 'industry_fintech', 'industry_gaming', 'industry_healthcare', 'industry_logistics', 'industry_media', 'industry_saas_b2b', 'company_size_Medium', 'company_size_Small', 'customer_tier_Enterprise', 'customer_tier_Plus', 'product_area_auth', 'product_area_billing', 'product_area_data_pipeline', 'product_area_mobile', 'product_area_notifications']


# **Balanceo de clase Target: priority**

In [ ]:
#29 verificamos frecuencias de priority
print("Distribución de la columna 'priority':")
display(dc['priority'].value_counts())
print("\nDistribución de la columna 'priority' (%):")
display(dc['priority'].value_counts(normalize=True) * 100)

Distribución de la columna 'priority':


,count
priority,
low,25000
medium,17500
high,7500



Distribución de la columna 'priority' (%):


,proportion
priority,
low,50.0
medium,35.0
high,15.0


**Conclusión:** Aplicar Classs-Weight para ecitar que el modelo le de preferencia a low, cuando los importantes son high

In [ ]:

#30 balanceamos priority con class_weight
from sklearn.utils import class_weight
import numpy as np

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(dc['priority'].astype(str)), # Convert unique classes to numpy array of strings
    y=dc['priority'].astype(str) # Also convert y to strings to match classes
)

# Create a dictionary of class weights
class_weight_dict = {
    cls: weight for cls, weight in zip(np.unique(dc['priority'].astype(str)), class_weights) # Use unique classes as strings for dictionary keys
}

print("Class weights for 'priority':")
display(class_weight_dict)

Class weights for 'priority':


{'high': np.float64(2.2222222222222223),
 'low': np.float64(0.6666666666666666),
 'medium': np.float64(0.9523809523809523)}